In [12]:
import requests
from bs4 import BeautifulSoup
import time
import numpy
import pandas as pd

## Scraping Event Page

In [2]:
## Store in the list
base_url = 'https://visitseattle.org/events/page/'
detail_links = [] 

for i in range(1, 42):
    url = base_url + str(i)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    selector = 'div.search-result-preview > div > h3 > a'
    detail_eles = soup.select(selector)
    # detail_list.append([x['href'] for x in detail_eles])
    for ele in detail_eles:
        detail_links.append(ele['href'])

In [3]:
# save the detail_links into pickle
import pickle
with open('detail_links.pkl', 'wb') as f:
    pickle.dump(detail_links, f)
# read the pickle
with open('detail_links.pkl', 'rb') as f:
    detail_links = pickle.load(f)

In [4]:
## Print out the list
for link in detail_links:
    print(link)

https://visitseattle.org/events/john-cole-peter-de-lory/
https://visitseattle.org/events/the-puppet-masters-making-fools-of-us-all/
https://visitseattle.org/events/whim-whim-winter-24/
https://visitseattle.org/events/compania-nacional-de-danza/
https://visitseattle.org/events/metropolis/
https://visitseattle.org/events/andy-coe-band/
https://visitseattle.org/events/align/
https://visitseattle.org/events/billy-idol-and-duran-duran-tribute-night/
https://visitseattle.org/events/crack-sabbath/
https://visitseattle.org/events/harlem-globetrotters/
https://visitseattle.org/events/inked-up-conversation-with-john-ellis/
https://visitseattle.org/events/jukebox-quartet-arcobaleno-strings/
https://visitseattle.org/events/pike-place-market-local-appreciation-day/
https://visitseattle.org/events/seattle-baroque-orchestra-party-bohemienne/
https://visitseattle.org/events/seattle-thunderbirds-vs-everett-silvertips-2/
https://visitseattle.org/events/the-budos-band/
https://visitseattle.org/events/the

## Scrape Detail Page

In [14]:
event_list = []

for event in detail_links:
    res = requests.get(event)
    soup = BeautifulSoup(res.text, 'html.parser')
    name_selector = '.medium-6.columns.event-top > h1'
    date_selector = '.medium-6.columns.event-top > h4 > span:nth-child(1)'
    location_selector = '.medium-6.columns.event-top > h4 > span:nth-child(2)'
    type_selector = '.medium-6.columns.event-top > a:nth-child(3)'
    region_selector = '.medium-6.columns.event-top > a:nth-child(4)'

    name = soup.select(name_selector)
    date = soup.select(date_selector)
    location = soup.select(location_selector)
    Type = soup.select(type_selector)
    region = soup.select(region_selector)
    event_list.append({'Name': name[0].text, 'Date': date[0].text, 'Location': location[0].text, 'Type': Type[0].text, 'Region': region[0].text})
    time.sleep(1)

df = pd.DataFrame(event_list)
df.to_csv('event_data.csv', index=False)

    

## Weather API

In [47]:
Location_list = [event['Location'] for event in event_list]
Location_list


[' Harris Harvey Gallery',
 ' Gallery 110',
 ' Cornish Playhouse at Seattle Center',
 ' Meany Center for the Performing Arts',
 ' Benaroya Hall',
 ' Nectar Lounge',
 ' High Dive',
 ' Sunset Tavern',
 ' Sea Monster Lounge',
 ' Climate Pledge Arena',
 ' Bainbridge Island Museum of Art',
 ' The Royal Room',
 ' Pike Place Market',
 ' Town Hall Seattle',
 ' accesso ShoWare Center',
 ' The Showbox',
 ' Opera Center',
 ' National Nordic Museum',
 ' Frye Art Museum',
 ' Fountainhead Gallery',
 ' Market Theatre',
 ' The Triple Door',
 ' 12th Avenue Arts',
 " Dimitriou's Jazz Alley",
 ' Tacoma Dome',
 ' Sea Monster Lounge',
 ' Bainbridge Island Museum of Art',
 ' Benaroya Hall',
 ' The Royal Room',
 ' National Nordic Museum',
 ' Burke Museum',
 ' Climate Pledge Arena',
 ' accesso ShoWare Center',
 ' Tractor Tavern',
 ' The Crocodile',
 ' Marion Oliver McCaw Hall',
 ' The Triple Door',
 ' High Dive',
 ' accesso ShoWare Center',
 ' Henry Art Gallery',
 ' Henry Art Gallery',
 " Dimitriou's Jazz All

In [48]:
map_url = "https://nominatim.openstreetmap.org/search"
query_params = {
    "q": Location_list[1],
    "format": "jsonv2"
}
res = requests.get(base_url, params=query_params)
map_dict = res.json()
map_dict

[{'place_id': 312937396,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'node',
  'osm_id': 2351711856,
  'lat': '47.6012826',
  'lon': '-122.330069',
  'category': 'shop',
  'type': 'art',
  'place_rank': 30,
  'importance': 9.99999999995449e-06,
  'addresstype': 'shop',
  'name': 'Gallery 110',
  'display_name': 'Gallery 110, 110, 3rd Avenue South, Japantown, Chinatown/International District, Seattle, King County, Washington, 98104, United States',
  'boundingbox': ['47.6012326', '47.6013326', '-122.3301190', '-122.3300190']}]

In [53]:
latitude = map_dict[0]['lat']
longitude = map_dict[0]['lon']
place = latitude, longitude
place

('47.6012826', '-122.330069')

In [56]:
weather_url = "https://api.weather.gov/points/{},{}".format(latitude, longitude)
print(weather_url)
res = requests.get(weather_url)
point_dict = res.json()
forcast_url = point_dict['properties']['forecast']
res = requests.get(forcast_url)
weather_dict = res.json()

https://api.weather.gov/points/47.6012826,-122.330069


In [57]:

weather = weather_dict['properties']['periods']
for i in weather:
    if 'Night' not in i['name'] and 'night' not in i['name']:
        print(i['name'], i['temperature'], 'F', i['shortForecast'])


Saturday 54 F Rain
Sunday 57 F Light Rain
Monday 58 F Light Rain Likely
Tuesday 58 F Chance Light Rain
Wednesday 54 F Light Rain
Thursday 51 F Light Rain
Friday 47 F Light Rain


In [22]:
# Option 1: just string manipulation
base_url = "https://nominatim.openstreetmap.org/search.php"
query_params_str = "?q=Wallingford%2C+Seattle&format=jsonv2"
ful_url = base_url + query_params_str

# Option 2: use dictionary to represent query params
#           use requests.get(url, params=query_params) to attach the query param dict
query_params = {
    "q": "Wallingford, Seattle",
    "format": "jsonv2"
}

res = requests.get(base_url, params=query_params)
res.json()

[{'place_id': 312844859,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'node',
  'osm_id': 150973716,
  'lat': '47.6594631',
  'lon': '-122.3343417',
  'category': 'place',
  'type': 'suburb',
  'place_rank': 19,
  'importance': 0.3131791431958073,
  'addresstype': 'suburb',
  'name': 'Wallingford',
  'display_name': 'Wallingford, Seattle, King County, Washington, 98015, United States',
  'boundingbox': ['47.6394631', '47.6794631', '-122.3543417', '-122.3143417']}]